# 🔴 RetroPGF3 Alternate Scoring Scenarios

Data available here:
<br>https://github.com/ethereum-optimism/op-analytics/blob/main/rpgf/rpgf3/results/anonymized_project_votes.csv

In [1]:
import numpy as np
import pandas as pd
import warnings

TOKENS = 30_000 # thousands

In [2]:
# load a local copy of the data
df = pd.read_csv("../data/anonymized_project_votes.csv")

# shorten name and remove special characters
def clean_name(string):
    string = "".join([s for s in string if s.isalnum() or s.isspace()])
    string = string.replace("  "," ")
    string = string.replace("Department of Decentralization", "Dept of Decentral")
    if len(string) > 20:
        string = " ".join(string[:15].split(" ")[:-1])
    string = string.strip("")
    return string
df['project_name'] = df['project_name'].apply(clean_name)

# turn the vote amounts into a list of values
df['amount'] = df['amount'].apply(eval)

# run some basic stats analysis
df['Median'] = round(df['amount'].apply(lambda amt: np.median(amt)))
df['Median (exclude 0s)'] = round(df['amount'].apply(lambda amt: np.median([x for x in amt if x])))
df['Mean'] = round(df['amount'].apply(lambda amt: np.mean(amt)))
df['Quadratic'] = df['amount'].apply(lambda x: sum(np.sqrt(x)))

# remove projects that didn't meet quorum
df = df[(df['number_of_votes'] >= 17) & (df['Median'] >= 1500)]

# remove columns we don't need
df.drop(columns=['amount', 'number_of_votes'], inplace=True)
df.set_index('project_name', inplace=True)

# calculate the alternate scores
df = round(df / df.sum(axis=0) * TOKENS,0).applymap(int)
df['variance'] = df.std(axis=1) / df.mean(axis=1)
low = df['variance'].quantile(.2)

# create some categories
df['category'] = df.idxmax(axis=1)
df['category'] = df.apply(lambda x: "No difference" if x['variance'] < low else x['category'], axis=1)
df['category'] = df.apply(
    lambda x: 
    "Quadratic (longtail)" if ((x['Quadratic'] < 25) and (x['category'] == 'Quadratic')) 
    else x['category'], axis=1)

df

,Median,Median (exclude 0s),Mean,Quadratic,variance,category
project_name,,,,,,
0xDeFiTH,9,10,6,10,0.216339,Median (exclude 0s)
0xKhmer,24,23,15,16,0.238705,Median
0xSalman,25,24,15,15,0.278481,Median
0xs34n,25,24,25,28,0.067924,No difference
1W3,20,19,18,20,0.049736,No difference
...,...,...,...,...,...,...
wslyvh,49,48,37,60,0.193785,Quadratic
zerokn0wledge,12,11,14,16,0.167348,Quadratic (longtail)
TurboETH,49,48,52,61,0.112687,No difference


In [3]:
df.to_csv("../data/2024-02-13_RetroPGF3_AlternateScoringScenarios.csv")